# 20201206：使用神经网络实现回归任务

## 作业要求
假设数据集 D={(X, Y)} 包含 N 个元组(X, Y)，其中 X 为输入，维度为 D_in；Y为输出，维度为 D_out。请你构建一个两层的神经网络，实现从 X 到 Y 的回归任务。

**截止时间：2020年12月8日 周二23:59**

In [1]:
import numpy as np
import torch

# N是数据集的大小
# D_in是输入维度，H是隐藏层维度，D_out是输出维度
N = 64
D_in, H, D_out = 1000, 100, 10

# 创建随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 反向传播中使用到的学习率参数
learning_rate = 1e-6

### 方法1 实现反向传播算法

1. 参考周五ppt内容，给出下列回归问题的计算图。
2. 请补全需要填空部分的代码，手动实现梯度的计算

参考内容：relu激活函数的导数推导https://blog.csdn.net/obrightlamp/article/details/84326978

In [2]:
# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)
for t in range(500):
    # 前向传播：计算输出值 y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    ## 1. 计算并打印平方损失
    # TODO
    y_loss=((y_pred-y)**2).sum()
    print(t,y_loss)
    ## 2.反向传播
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    # 计算 w1 的梯度
    # TODO
    grad_w1=x.T@((grad_y_pred@w2.T)*(h>0))
    ## 3.更新权重 w1 和 w2
    # TODO
    w1-=grad_w1*learning_rate
    w2-=grad_w2*learning_rate

0 33681168.8523612
1 32960478.476006877
2 36942680.46977167
3 38861548.8700012
4 33714722.67844294
5 22432593.712130755
6 11793057.218999516
7 5526994.653633564
8 2725489.6775141275
9 1563618.0517414908
10 1053410.775040931
11 792522.044358729
12 634193.2249409843
13 524077.8363076663
14 440704.36681780417
15 374421.7476497939
16 320422.55503852887
17 275760.87291588983
18 238478.5945684846
19 207134.9120469901
20 180645.10530320063
21 158136.9885287583
22 138907.65617667075
23 122417.39458442338
24 108234.35491232577
25 95977.73055744274
26 85337.74316938448
27 76073.20597499525
28 67981.09906948027
29 60886.9708137859
30 54651.01701610812
31 49155.700438315725
32 44309.59972232595
33 40016.92540508836
34 36203.37928151153
35 32808.73529776701
36 29779.816295090237
37 27074.421796291852
38 24651.6733417174
39 22478.283551847344
40 20522.82881222113
41 18760.933695157182
42 17170.852743887102
43 15733.931699233977
44 14433.528420958748
45 13255.225074974867
46 12187.04371901021
47 1121

### 方法2：使用自动求导
请补全需要填空部分的代码（使用pytorch进行自动求导）

对比1和2的结果

In [3]:
dtype = torch.float
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 1. 使用上一题中x, y的数值构建x_tensor, y_tensor
# TODO
x_tensor=torch.tensor(x,device=device,dtype=dtype)
y_tensor=torch.tensor(y,device=device,dtype=dtype)

# 随机初始化权重
# 设置requires_grad = True表示我们想要计算梯度
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)
for t in range(500):
    # 前向传播：使用tensors上的操作计算输出值 y
    # 由于 w1 和 w2 有requires_grad=True，涉及这些张量的操作将让PyTorch构建计算图，
    # 从而允许自动计算梯度。由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    ## 2. 补充y_pred的计算
    # TODO
    y_pred=torch.maximum((x_tensor@w1),torch.zeros(N,H,device=device,dtype=dtype))@w2

    # 使用Tensors上的操作计算并打印 loss。
    # loss.item() 得到这个张量对应的python数值
    loss = (y_pred - y_tensor).pow(2).sum()
    print(t, loss.item())

    ## 3.补充自动求导backward
    # TODO
    loss.backward()

    # 使用梯度下降更新权重。对于这一步，我们只想对 w1 和 w2 的值进行原地改变
    # 不想为更新阶段构建计算图
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        ## 4.补充参数更新的代码
        # TODO
        w1.sub_(w1.grad,alpha=learning_rate)
        w2.sub_(w2.grad,alpha=learning_rate)
        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 28156636.0
1 21747104.0
2 17748808.0
3 14152216.0
4 10714157.0
5 7680085.0
6 5316839.0
7 3627964.0
8 2495991.75
9 1757940.375
10 1279355.5
11 964536.375
12 752428.75
13 604382.5
14 497179.71875
15 416844.0
16 354620.9375
17 305116.0625
18 264829.0
19 231497.125
20 203507.03125
21 179715.484375
22 159395.84375
23 141911.953125
24 126733.375
25 113487.921875
26 101877.71875
27 91693.421875
28 82707.75
29 74777.3984375
30 67733.53125
31 61465.55078125
32 55864.8984375
33 50852.0390625
34 46356.140625
35 42314.0546875
36 38672.1875
37 35387.46484375
38 32417.427734375
39 29728.248046875
40 27291.70703125
41 25080.138671875
42 23071.107421875
43 21241.28125
44 19574.0390625
45 18052.703125
46 16662.078125
47 15391.369140625
48 14226.931640625
49 13159.3681640625
50 12180.017578125
51 11281.259765625
52 10455.3125
53 9695.7509765625
54 8996.599609375
55 8353.7138671875
56 7760.85546875
57 7214.0693359375
58 6709.43701171875
59 6243.48974609375
60 5812.1220703125
61 5413.07275390625
62 5043